In [1]:
%%init_spark
launcher.num_executors = 4
launcher.executor_cores = 2
launcher.driver_memory = "4g"

launcher.conf.set("spark.sql.catalogImplementation", "hive")

In [2]:
import java.nio.file.Files
import java.nio.file.Paths
import scala.sys.process._

val CWD = System.getProperty("user.dir")
val dataFile = CWD + "/mini_sparkify_event_data.json.gz"

if (!Files.exists(Paths.get(dataFile))) { 
    println("download file...")
    val output = "wget https://github.com/gnosia93/spark-on-eks/raw/main/dataset/mini_sparkify_event_data.json.gz".!! // Captures the output
}


Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.23:4040
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1675558971623)
SparkSession available as 'spark'


import java.nio.file.Files
import java.nio.file.Paths
import scala.sys.process._
CWD: String = /Users/soonbeom/ml/scala-spark
dataFile: String = /Users/soonbeom/ml/scala-spark/mini_sparkify_event_data.json.gz


In [3]:
import java.nio.file.Files
import java.nio.file.Paths
import scala.sys.process._

val CWD = System.getProperty("user.dir")
val dataFile = CWD + "/mini_sparkify_event_data.json.gz"

if (!Files.exists(Paths.get(dataFile))) { 
    println("download file...")
    val output = "wget https://github.com/gnosia93/spark-on-eks/raw/main/dataset/mini_sparkify_event_data.json.gz".!! // Captures the output
}


import java.nio.file.Files
import java.nio.file.Paths
import scala.sys.process._
CWD: String = /Users/soonbeom/ml/scala-spark
dataFile: String = /Users/soonbeom/ml/scala-spark/mini_sparkify_event_data.json.gz


In [4]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
    .builder()
    .appName("Sparky Analysis")
    .getOrCreate()

spark.getClass()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6f7d1a2
res2: Class[_ <: org.apache.spark.sql.SparkSession] = class org.apache.spark.sql.SparkSession


In [5]:
val df = spark.read.json("mini_sparkify_event_data.json.gz")
df.printSchema()
println("row count: " + df.count())
df.show(10)

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

row count: 286500
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSessi

df: org.apache.spark.sql.DataFrame = [artist: string, auth: string ... 16 more fields]


In [6]:
df.createOrReplaceTempView("event_data")

In [7]:
df.describe("itemInSession", "length", "registration", "sessionId", "status", "ts").show()

+-------+------------------+-----------------+--------------------+-----------------+------------------+--------------------+
|summary|     itemInSession|           length|        registration|        sessionId|            status|                  ts|
+-------+------------------+-----------------+--------------------+-----------------+------------------+--------------------+
|  count|            286500|           228108|              278154|           286500|            286500|              286500|
|   mean|114.41421291448516|249.1171819778458|1.535358834084427...|1041.526554973822|210.05459685863875|1.540956889810483...|
| stddev|129.76726201140994|99.23517921058361| 3.291321616327586E9|726.7762634630741| 31.50507848842214|1.5075439608226302E9|
|    min|                 0|          0.78322|       1521380675000|                1|               200|       1538352117000|
|    max|              1321|       3024.66567|       1543247354000|             2474|               404|       1543799

In [8]:
df.groupBy("page").count().show()
df.select("userid").filter("registration is null").count()

+--------------------+------+
|                page| count|
+--------------------+------+
|              Cancel|    52|
|    Submit Downgrade|    63|
|         Thumbs Down|  2546|
|                Home| 14457|
|           Downgrade|  2055|
|         Roll Advert|  3933|
|              Logout|  3226|
|       Save Settings|   310|
|Cancellation Conf...|    52|
|               About|   924|
| Submit Registration|     5|
|            Settings|  1514|
|               Login|  3241|
|            Register|    18|
|     Add to Playlist|  6526|
|          Add Friend|  4277|
|            NextSong|228108|
|           Thumbs Up| 12551|
|                Help|  1726|
|             Upgrade|   499|
+--------------------+------+
only showing top 20 rows



res6: Long = 8346


### 상관관계 분석 ###

 ```
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 ```

In [9]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.stat.Correlation

/*
val assembler = new VectorAssembler()
  .setInputCols(Array("itemInSession", "registration", "sessionId", "status", "ts"))
  .setOutputCol("vectors")

val transformed = assembler.transform(df)
val corr = Correlation.corr(transformed, "vectors").head
println(s"Pearson correlation matrix:\n $corr")
*/

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.stat.Correlation


### Temp View 생성 ###

In [10]:
df.createOrReplaceTempView("event_data")  // Temp View 생성

val grpByUserId = spark.sql("""
            SELECT userId, 
                   count(sessionId) as numSessions, 
                   count(distinct song) as numSongs, 
                   count(distinct artist) as numArtists,
                   floor(sum(length)) as playtime, 
                   from_unixtime(max(registration/1000)) as registDate,
                   current_timestamp
            FROM event_data
            GROUP BY userId
          """)
grpByUserId.show(5)

+------+-----------+--------+----------+--------+-------------------+--------------------+
|userId|numSessions|numSongs|numArtists|playtime|         registDate| current_timestamp()|
+------+-----------+--------+----------+--------+-------------------+--------------------+
|100010|        381|     269|       252|   66940|2018-09-27 11:45:40|2023-02-05 10:03:...|
|200002|        474|     378|       339|   94008|2018-09-07 06:38:26|2023-02-05 10:03:...|
|   125|         11|       8|         8|    2089|2018-08-02 05:58:59|2023-02-05 10:03:...|
|    51|       2464|    1854|      1385|  523275|2018-09-28 05:43:07|2023-02-05 10:03:...|
|   124|       4825|    3339|      2232| 1012312|2018-07-22 10:52:15|2023-02-05 10:03:...|
+------+-----------+--------+----------+--------+-------------------+--------------------+
only showing top 5 rows



grpByUserId: org.apache.spark.sql.DataFrame = [userId: string, numSessions: bigint ... 5 more fields]


### EDA - Dealing with Missing Value (누락값 다루기) ###

파이썬 인터트리터에서 scala 데이터 프레임을 읽기 위해, createOrReplaceTempView 로 임시테이블(event_data)을 만든다.  
%%python 매직키워드를 사용하여 python 인터 프리터로 전환하고, seaborn 으로 hitmap 을 그린다.

In [11]:
%%python

import matplotlib.pyplot as plt
import seaborn as sns

df = spark.sql("select * from event_data")
plt.figure(figsize=(18,6))
sns.heatmap(df.toPandas().isnull(),cbar=False)
plt.show()

In [12]:
df.select("*").filter("registration is null").show(10)
df.select("*").filter("userId = ''").show(10)

+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method| page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|          100|    null|  null| free|    null|   GET| Home|        null|        8|null|   200|1538355745000|     null|      |
|  null|Logged Out|     null|  null|          101|    null|  null| free|    null|   GET| Help|        null|        8|null|   200|1538355807000|     null|      |
|  null|Logged Out|     null|  null|          102|    null|  null| free|    null|   GET| Home|        null|        8|null|   200|1538355841000|     null|      |
|  null|Logged Out|     null|  nul

In [13]:
df.filter(df("artist").isNull).select("page") 
    .groupBy("page").count
    .orderBy($"count".desc)
    .show(df.count.toInt, false)

df.filter(df("artist").isNotNull).select("page") 
    .groupBy("page").count
    .orderBy($"count".desc)
    .show(df.count.toInt, false)

+-------------------------+-----+
|page                     |count|
+-------------------------+-----+
|Home                     |14457|
|Thumbs Up                |12551|
|Add to Playlist          |6526 |
|Add Friend               |4277 |
|Roll Advert              |3933 |
|Login                    |3241 |
|Logout                   |3226 |
|Thumbs Down              |2546 |
|Downgrade                |2055 |
|Help                     |1726 |
|Settings                 |1514 |
|About                    |924  |
|Upgrade                  |499  |
|Save Settings            |310  |
|Error                    |258  |
|Submit Upgrade           |159  |
|Submit Downgrade         |63   |
|Cancel                   |52   |
|Cancellation Confirmation|52   |
|Register                 |18   |
|Submit Registration      |5    |
+-------------------------+-----+

+--------+------+
|page    |count |
+--------+------+
|NextSong|228108|
+--------+------+



## Feature Engineering ##

데이테에 대한 경험과 도메인 지식을 기반으로 firstName, lastName, method 및 userAgent 칼럼은 모델링에 포함하지 않기로 한다.   
보통 이런류의 피처는 예측에 영향을 주지 않는다. 또한 artist, location, song 및 status 역시 제외하도록 한다.

In [14]:
val drop_df = df.drop("firstName", "lastName", "method", "userAgent", "artist", "location", "song", "status")
drop_df.printSchema()
drop_df.show()

root
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userId: string (nullable = true)

+---------+------+-------------+---------+-----+---------------+-------------+---------+-------------+------+
|     auth|gender|itemInSession|   length|level|           page| registration|sessionId|           ts|userId|
+---------+------+-------------+---------+-----+---------------+-------------+---------+-------------+------+
|Logged In|     M|           50|277.89016| paid|       NextSong|1538173362000|       29|1538352117000|    30|
|Logged In|     M|           79|236.09424| free|       NextSong|1538331630000|        8|1538352180000|     9|
|Logged In|     M|           51| 282.8273| paid|       NextSong|1

drop_df: org.apache.spark.sql.DataFrame = [auth: string, gender: string ... 8 more fields]


### 1. Define Churned User(이탈 사용자 정의) ###
Churned User 는 1 대 3 정도의 비율을 가지고 있는 것을 알 수 있다. (52명이 탈퇴함)

In [15]:
import org.apache.spark.sql.functions.udf
val func = udf((s:String) => if(s == "Cancellation Confirmation") 1 else 0)

val churn_df = drop_df.withColumn("churn", func($"page"))
churn_df.show(10)

+---------+------+-------------+---------+-----+---------------+-------------+---------+-------------+------+-----+
|     auth|gender|itemInSession|   length|level|           page| registration|sessionId|           ts|userId|churn|
+---------+------+-------------+---------+-----+---------------+-------------+---------+-------------+------+-----+
|Logged In|     M|           50|277.89016| paid|       NextSong|1538173362000|       29|1538352117000|    30|    0|
|Logged In|     M|           79|236.09424| free|       NextSong|1538331630000|        8|1538352180000|     9|    0|
|Logged In|     M|           51| 282.8273| paid|       NextSong|1538173362000|       29|1538352394000|    30|    0|
|Logged In|     M|           80|262.71302| free|       NextSong|1538331630000|        8|1538352416000|     9|    0|
|Logged In|     M|           52|223.60771| paid|       NextSong|1538173362000|       29|1538352676000|    30|    0|
|Logged In|     M|           81|208.29995| free|       NextSong|15383316

import org.apache.spark.sql.functions.udf
func: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4798/0x000000080197a840@7e8d11d6,IntegerType,List(Some(class[value[0]: string])),Some(class[value[0]: int]),None,false,true)
churn_df: org.apache.spark.sql.DataFrame = [auth: string, gender: string ... 9 more fields]


In [16]:
import org.apache.spark.sql.expressions.Window

val w1 = Window.partitionBy("userId")
var user_df = churn_df.select("userId", "churn", "gender", "level")
                    .withColumn("churned_user", sum("churn").over(w1))
                    .dropDuplicates("userId")
                    .drop("churn")

user_df = user_df.na.drop(Seq("userId", "gender", "level"))
user_df.groupBy("churned_user").count().show()

+------------+-----+
|churned_user|count|
+------------+-----+
|           0|  173|
|           1|   52|
+------------+-----+



import org.apache.spark.sql.expressions.Window
w1: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2a5eb2ea
user_df: org.apache.spark.sql.DataFrame = [userId: string, gender: string ... 2 more fields]
user_df: org.apache.spark.sql.DataFrame = [userId: string, gender: string ... 2 more fields]


In [17]:
churn_df.groupBy("userId").count.show()
churn_df.filter("userId == 125").show(100, false)

+------+-----+
|userId|count|
+------+-----+
|100010|  381|
|200002|  474|
|   125|   11|
|    51| 2464|
|   124| 4825|
|     7|  201|
|    54| 3437|
|    15| 2278|
|   155| 1002|
|   132| 2304|
|   154|  118|
|100014|  310|
|   101| 2149|
|    11|  848|
|   138| 2469|
|300017| 4428|
|    29| 3603|
|    69| 1342|
|100021|  319|
|    42| 4257|
+------+-----+
only showing top 20 rows

+---------+------+-------------+---------+-----+-------------------------+-------------+---------+-------------+------+-----+
|auth     |gender|itemInSession|length   |level|page                     |registration |sessionId|ts           |userId|churn|
+---------+------+-------------+---------+-----+-------------------------+-------------+---------+-------------+------+-----+
|Logged In|M     |0            |337.91955|free |NextSong                 |1533157139000|174      |1539317144000|125   |0    |
|Logged In|M     |1            |230.03383|free |NextSong                 |1533157139000|174      |153931748100

### 2. 범주형 피처 인코딩 ###
범주형 피처의 경우 먼저 레이블 인코딩이 필요합니다(단순히 각 값을 숫자로 변환). 기계 학습 모델에 따라 이러한 숫자를 더미 변수(예: 원-핫 인코딩)로 추가 인코딩해야 할 수도 있습니다.  
Spark에서 StringIndexer는 레이블 인코딩 부분을 수행합니다.

In [18]:
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}

val indexer = new StringIndexer()
    .setInputCols(List("gender", "level").toArray)
    .setOutputCols(List("genderIndex", "levelIndex").toArray)
val user_df_le = indexer.fit(user_df).transform(user_df)

user_df_le.show(10)

+------+------+-----+------------+-----------+----------+
|userId|gender|level|churned_user|genderIndex|levelIndex|
+------+------+-----+------------+-----------+----------+
|    10|     M| paid|           0|        0.0|       1.0|
|   100|     M| paid|           0|        0.0|       1.0|
|100001|     F| free|           1|        1.0|       0.0|
|100002|     F| paid|           0|        1.0|       1.0|
|100003|     F| free|           1|        1.0|       0.0|
|100004|     F| free|           0|        1.0|       0.0|
|100005|     M| free|           1|        0.0|       0.0|
|100006|     F| free|           1|        1.0|       0.0|
|100007|     F| paid|           1|        1.0|       1.0|
|100008|     F| paid|           0|        1.0|       1.0|
+------+------+-----+------------+-----------+----------+
only showing top 10 rows



import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_cfcae064bc81
user_df_le: org.apache.spark.sql.DataFrame = [userId: string, gender: string ... 4 more fields]


In [19]:
user_df_le.groupBy("genderIndex").avg("churned_user").show()
user_df_le.groupBy("churned_user").avg("levelIndex").show()

+-----------+-------------------+
|genderIndex|  avg(churned_user)|
+-----------+-------------------+
|        0.0| 0.2644628099173554|
|        1.0|0.19230769230769232|
+-----------+-------------------+

+------------+-------------------+
|churned_user|    avg(levelIndex)|
+------------+-------------------+
|           0|0.23121387283236994|
|           1|0.15384615384615385|
+------------+-------------------+



gender 와 level 피처에 대해 원핫 인코딩을 적용한다.

In [20]:
var encoder = new OneHotEncoder()
      .setInputCols(List("genderIndex", "levelIndex").toArray)
      .setOutputCols(List("genderVec", "levelVec").toArray)

val user_df_oh = encoder.fit(user_df_le).transform(user_df_le)
user_df_oh.show()

+------+------+-----+------------+-----------+----------+-------------+-------------+
|userId|gender|level|churned_user|genderIndex|levelIndex|    genderVec|     levelVec|
+------+------+-----+------------+-----------+----------+-------------+-------------+
|    10|     M| paid|           0|        0.0|       1.0|(1,[0],[1.0])|    (1,[],[])|
|   100|     M| paid|           0|        0.0|       1.0|(1,[0],[1.0])|    (1,[],[])|
|100001|     F| free|           1|        1.0|       0.0|    (1,[],[])|(1,[0],[1.0])|
|100002|     F| paid|           0|        1.0|       1.0|    (1,[],[])|    (1,[],[])|
|100003|     F| free|           1|        1.0|       0.0|    (1,[],[])|(1,[0],[1.0])|
|100004|     F| free|           0|        1.0|       0.0|    (1,[],[])|(1,[0],[1.0])|
|100005|     M| free|           1|        0.0|       0.0|(1,[0],[1.0])|(1,[0],[1.0])|
|100006|     F| free|           1|        1.0|       0.0|    (1,[],[])|(1,[0],[1.0])|
|100007|     F| paid|           1|        1.0|       1

encoder: org.apache.spark.ml.feature.OneHotEncoder = oneHotEncoder_cb61fb3f2a83
user_df_oh: org.apache.spark.sql.DataFrame = [userId: string, gender: string ... 6 more fields]
